# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        #print(root, dirs, files)
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data');


In [5]:
filepath = song_files[0]
print(filepath)


/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json


In [6]:
df = pd.read_json(filepath, lines=True)
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


In [7]:
df['year'] = df['year'].apply(lambda x: x if x != 0 else None)
df = df.replace({pd.np.nan: None, "": None})
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,None,None,None,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),None


In [8]:
df = pd.read_json(path_or_buf=filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = list(df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0])
print(song_data)

['SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', None, 266.39628]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
try:
    cur.execute(song_table_insert.strip(), song_data)
except psycopg.Error as e:
    print(e)
conn.commit()

In [10]:
song_table_insert.strip()

'INSERT INTO songs (song_id, title, artist_id, year, duration)\nVALUES (%s, %s, %s, %s, %s) ON CONFLICT DO NOTHING'

In [11]:
cur.execute("select * from songs")
row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()


('SOUDSGM12AC9618304', 'Insatiable (Instrumental Version)', 'ARNTLGG11E2835DDB9', None, Decimal('266.39628'))


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = list(df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0])
artist_data

['ARNTLGG11E2835DDB9', 'Clp', None, None, None]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [13]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [14]:
log_files = get_files("data/log_data")

In [15]:
filepath = log_files[11] 
print(filepath)

/home/workspace/data/log_data/2018/11/2018-11-18-events.json


In [16]:
df = pd.concat([pd.read_json(log, lines = 'true') for log in log_files])
print(len(df))

df.head()

8056


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Mitch Ryder & The Detroit Wheels,Logged In,Tegan,F,65,Levine,205.03465,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Jenny Take A Ride (LP Version),200,1543363215796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
1,The Spill Canvas,Logged In,Tegan,F,66,Levine,358.03383,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,The TIde (LP Version),200,1543363420796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
2,Mogwai,Logged In,Tegan,F,67,Levine,571.19302,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Two Rights Make One Wrong,200,1543363778796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
3,Spor,Logged In,Tegan,F,68,Levine,380.34240,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Way Of The Samurai,200,1543364349796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
4,DJ Dizzy,Logged In,Tegan,F,69,Levine,221.15220,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Sexy Bitch,200,1543364729796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80


In [17]:
df = pd.read_json(filepath, lines=True,)
#df.tail()
print(len(df))

157


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 18 columns):
artist           138 non-null object
auth             157 non-null object
firstName        153 non-null object
gender           153 non-null object
itemInSession    157 non-null int64
lastName         153 non-null object
length           138 non-null float64
level            157 non-null object
location         153 non-null object
method           157 non-null object
page             157 non-null object
registration     153 non-null float64
sessionId        157 non-null int64
song             138 non-null object
status           157 non-null int64
ts               157 non-null int64
userAgent        153 non-null object
userId           157 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 22.2+ KB


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Rokia TraorÃÂ©,Logged In,Stefany,F,0,White,274.88608,free,"Lubbock, TX",PUT,NextSong,1.540708e+12,693,Zen,200,2018-11-18 02:33:21.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83


In [20]:
df['timestamp'] = pd.to_datetime(df['ts'], unit='ms')
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.day
df['week'] = df['timestamp'].dt.week
df['month'] = df['timestamp'].dt.month
df['year'] = df['timestamp'].dt.year
df['weekday'] = df['timestamp'].dt.weekday
df.tail()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userAgent,userId,timestamp,hour,day,week,month,year,weekday
152,Weezer,Logged In,Jacqueline,F,126,Lynch,139.65016,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,...,2018-11-18 21:00:36.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29,2018-11-18 21:00:36.796,21,18,46,11,2018,6
153,Gary Stewart,Logged In,Chloe,F,8,Cuevas,182.67383,paid,"San Francisco-Oakland-Hayward, CA",PUT,...,2018-11-18 21:01:35.796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49,2018-11-18 21:01:35.796,21,18,46,11,2018,6
154,None,Logged In,Jacqueline,F,127,Lynch,NaN,paid,"Atlanta-Sandy Springs-Roswell, GA",GET,...,2018-11-18 21:03:31.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29,2018-11-18 21:03:31.796,21,18,46,11,2018,6
155,None,Logged In,Aiden,M,0,Hess,NaN,free,"La Crosse-Onalaska, WI-MN",GET,...,2018-11-18 23:02:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",86,2018-11-18 23:02:41.796,23,18,46,11,2018,6
156,Travie McCoy,Logged In,Aiden,M,1,Hess,211.12118,free,"La Crosse-Onalaska, WI-MN",PUT,...,2018-11-18 23:02:44.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",86,2018-11-18 23:02:44.796,23,18,46,11,2018,6


In [21]:
df = df.loc[df['page'] == 'NextSong']

In [22]:
time_df = df.filter(items=['timestamp', 'hour', 'day', 'week', 'month', 'year', 'weekday'])
time_df.head()

,timestamp,hour,day,week,month,year,weekday
0,2018-11-18 02:33:21.796,2,18,46,11,2018,6
2,2018-11-18 03:22:04.796,3,18,46,11,2018,6
4,2018-11-18 09:39:04.796,9,18,46,11,2018,6
5,2018-11-18 10:18:11.796,10,18,46,11,2018,6
9,2018-11-18 13:16:01.796,13,18,46,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()
    

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df.filter(items=['userId', 'firstName', 'lastName', 'gender', 'level'])
user_df.head() 

,userId,firstName,lastName,gender,level
0,83,Stefany,White,F,free
2,40,Tucker,Garrison,M,free
4,40,Tucker,Garrison,M,free
5,79,James,Martin,M,free
9,29,Jacqueline,Lynch,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userAgent,userId,timestamp,hour,day,week,month,year,weekday
0,Rokia TraorÃÂ©,Logged In,Stefany,F,0,White,274.88608,free,"Lubbock, TX",PUT,...,2018-11-18 02:33:21.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83,2018-11-18 02:33:21.796,2,18,46,11,2018,6
2,Camila,Logged In,Tucker,M,1,Garrison,230.81751,free,"Oxnard-Thousand Oaks-Ventura, CA",PUT,...,2018-11-18 03:22:04.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",40,2018-11-18 03:22:04.796,3,18,46,11,2018,6
4,Carl Thomas,Logged In,Tucker,M,0,Garrison,196.67546,free,"Oxnard-Thousand Oaks-Ventura, CA",PUT,...,2018-11-18 09:39:04.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",40,2018-11-18 09:39:04.796,9,18,46,11,2018,6
5,N.E.R.D.,Logged In,James,M,0,Martin,242.99057,free,"Dallas-Fort Worth-Arlington, TX",PUT,...,2018-11-18 10:18:11.796,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,79,2018-11-18 10:18:11.796,10,18,46,11,2018,6
9,Lil Jon / The East Side Boyz / DJ Flexx,Logged In,Jacqueline,F,3,Lynch,285.30893,paid,"Atlanta-Sandy Springs-Roswell, GA",PUT,...,2018-11-18 13:16:01.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",29,2018-11-18 13:16:01.796,13,18,46,11,2018,6


In [27]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId', 'timestamp',
       'hour', 'day', 'week', 'month', 'year', 'weekday'],
      dtype='object')

In [28]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    print()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    #songplay_data = (0, pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    songplay_data = (pd.to_datetime(row.ts, unit='ms'), row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [11]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.